In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
import re
from tensorflow.python.keras import backend
from tensorflow.python.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [19]:
from google.colab import drive #Dataset mounted from google drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
df = pd.read_csv('/content/drive/MyDrive/anime_with_synopsis.csv')

In [21]:
df

,MAL_ID,Name,Score,Genders,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...
...,...,...,...,...,...
16209,48481,Daomu Biji Zhi Qinling Shen Shu,Unknown,"Adventure, Mystery, Supernatural",No synopsis information has been added to this...
16210,48483,Mieruko-chan,Unknown,"Comedy, Horror, Supernatural",ko is a typical high school student whose life...
16211,48488,Higurashi no Naku Koro ni Sotsu,Unknown,"Mystery, Dementia, Horror, Psychological, Supe...",Sequel to Higurashi no Naku Koro ni Gou .
16212,48491,Yama no Susume: Next Summit,Unknown,"Adventure, Slice of Life, Comedy",New Yama no Susume anime.


In [22]:
df.Genders.value_counts().head(15) #visualizing top 15 genres

Music                    790
Comedy                   668
Kids                     413
Kids, Music              234
Dementia                 228
Slice of Life            217
Music, Kids              201
Slice of Life, Comedy    171
Fantasy                  153
Drama                    136
Comedy, Kids             130
Comedy, Parody           120
Comedy, Slice of Life    119
Adventure, Fantasy       103
Drama, Kids               95
Name: Genders, dtype: int64

In [23]:
temp1=df[df["Genders"] == "Music"]
temp2=df[df["Genders"] == "Comedy"]
temp3=df[df["Genders"] == "Fantasy"]
temp4=df[df["Genders"] == "Drama"]
temp5=df[df["Genders"] == "Dementia"]

In [24]:
merged_df = pd.concat([temp1, temp2, temp3,temp4,temp5]) #merge only 5 genres for better accuracy

In [25]:
merged_df.Genders.value_counts()

Music       790
Comedy      668
Dementia    228
Fantasy     153
Drama       136
Name: Genders, dtype: int64

In [26]:
df=merged_df

In [27]:
df

,MAL_ID,Name,Score,Genders,sypnopsis
1359,1561,Lovedol: Lovely Idol,6.18,Music,"ovedol, or Lovely Idol, is a girl group consis..."
1973,2274,CLAMP in Wonderland,6.86,Music,short video set to music featuring many of CLA...
2321,2728,TWO-MIX: White Reflection,6.76,Music,"music video to Two-Mix's ""White Reflection""; t..."
2464,2938,Idol Densetsu Eriko,6.82,Music,"Eriko Tamura is Kousuke Tamura, the chairman o..."
2870,3685,4-Day Weekend,5.66,Music,usic video for the song Four-Day Weekend by a ...
...,...,...,...,...,...
15590,42281,Zap Cat,Unknown,Dementia,Completed psychedelic anime. As a personal wor...
15621,42354,Fusuma no Oku,Unknown,Dementia,The theme came from the Japanese unique folklo...
15782,42800,Wild Cleaner,Unknown,Dementia,station ID by Densuke28 for MTV ULTRAHITS.
15991,44059,Puparia,7.48,Dementia,Independent short animation by Shingo Tamagawa.


In [28]:
df=merged_df.reset_index(drop=True) #reset the starting index from 0

In [29]:
df

,MAL_ID,Name,Score,Genders,sypnopsis
0,1561,Lovedol: Lovely Idol,6.18,Music,"ovedol, or Lovely Idol, is a girl group consis..."
1,2274,CLAMP in Wonderland,6.86,Music,short video set to music featuring many of CLA...
2,2728,TWO-MIX: White Reflection,6.76,Music,"music video to Two-Mix's ""White Reflection""; t..."
3,2938,Idol Densetsu Eriko,6.82,Music,"Eriko Tamura is Kousuke Tamura, the chairman o..."
4,3685,4-Day Weekend,5.66,Music,usic video for the song Four-Day Weekend by a ...
...,...,...,...,...,...
1970,42281,Zap Cat,Unknown,Dementia,Completed psychedelic anime. As a personal wor...
1971,42354,Fusuma no Oku,Unknown,Dementia,The theme came from the Japanese unique folklo...
1972,42800,Wild Cleaner,Unknown,Dementia,station ID by Densuke28 for MTV ULTRAHITS.
1973,44059,Puparia,7.48,Dementia,Independent short animation by Shingo Tamagawa.


In [30]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [66]:
df = df.reset_index(drop=True)
replace_by_space = re.compile('[/(){}\[\]\|@,;]')
bad_symbols = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):

    text = text.lower() # lowercase text
    text = replace_by_space.sub(' ', text) # replace 'replace_by_space' symbols by space in text and substitute the matched string with space.
    text = bad_symbols.sub('', text) # remove symbols which are in 'bad_symbols' from text and substitute the matched string with nothing. 
    text = text.replace('x', '')
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwords from text
    return text
df['sypnopsis'] = df['sypnopsis'].apply(clean_text)
df['sypnopsis'] = df['sypnopsis'].str.replace('\d+', '')

AttributeError: ignored

In [67]:
df

,MAL_ID,Name,Score,Genders,sypnopsis
0,1561,Lovedol: Lovely Idol,6.18,Music,ovedol or lovely idol is a girl group consisti...
1,2274,CLAMP in Wonderland,6.86,Music,short video set to music featuring many of cla...
2,2728,TWO-MIX: White Reflection,6.76,Music,"music video to two-mi's ""white reflection""; th..."
3,2938,Idol Densetsu Eriko,6.82,Music,eriko tamura is kousuke tamura the chairman of...
4,3685,4-Day Weekend,5.66,Music,usic video for the song four-day weekend by a ...
...,...,...,...,...,...
1970,42281,Zap Cat,Unknown,Dementia,completed psychedelic anime. as a personal wor...
1971,42354,Fusuma no Oku,Unknown,Dementia,the theme came from the japanese unique folklo...
1972,42800,Wild Cleaner,Unknown,Dementia,station id by densuke for mtv ultrahits.
1973,44059,Puparia,7.48,Dementia,independent short animation by shingo tamagawa.


In [68]:
df['sypnopsis'].values

array(['ovedol or lovely idol is a girl group consisting of two generations of idols. the rd generation is about to make its debut but to their surprise they have been delayed! the vague reason is only that "they lack something." the manager and the rd gen. girls too must find the answer.',
       "short video set to music featuring many of clamp's most popular anime and manga characters (at the time of its production). the accompanying song tells of a wonderland filled with imagination magic and dreams. (source: ann)",
       'music video to two-mi\'s "white reflection"; the group in animated form notices a strange light and follows it to various locations.',
       ..., 'station id by densuke for mtv ultrahits.',
       'independent short animation by shingo tamagawa.',
       'tv show that is so unclear to whom and where it is broadcast. a strange room where the recording and editing is taking place. a middle-aged gentleman like a wa figure sits on a turntable and grabs the micropho

In [69]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 5000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)


In [70]:
X = tokenizer.texts_to_sequences(df['sypnopsis'].values)

AttributeError: ignored